# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686545


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:14,  2.66s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:54,  2.01s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:38,  1.47s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:28,  1.16s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:13<00:01,  2.93it/s]

Rendering models:  90%|█████████ | 27/30 [00:14<00:01,  2.99it/s]

Rendering models:  93%|█████████▎| 28/30 [00:14<00:00,  2.88it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.44it/s]

not-logged-in-bc5fdb9ea8d611e60b8b    0.000835
shirogane                             0.000335
olivia.barfield                       0.000361
Linda_J._Berkel                       0.000284
not-logged-in-742cef5e1a82d2ddd0fb    0.000638
mkehrli                               0.000288
lsautter                              0.000298
FelipedaFonseca                       0.000436
not-logged-in-41e9fe197dc7a2be8424    0.000471
Raven_Vasquez                         0.000877
not-logged-in-cd6a5cac0334cc279337    0.000520
pangeli5                              0.000362
rhondacard                            0.001322
Lavadude                              0.008688
ShirleyL                              0.000354
Bambura                               0.000396
kgenere                               0.000684
bboyle23                              0.001593
h4ranek                               0.000626
Fazal.H                               0.022948
not-logged-in-5c973654fb0968b87ca1    0.000323
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())